In [68]:
import pickle
import datetime
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer('uer/sbert-base-chinese-nli')
# cyclone/simcse-chinese-roberta-wwm-ext
# uer/sbert-base-chinese-nli 
# pe65374/xcoa-sbert-base-chinese-nli 


No sentence-transformers model found with name /Users/chenyangkang/.cache/torch/sentence_transformers/uer_sbert-base-chinese-nli. Creating a new one with MEAN pooling.


In [36]:
### deal date
def get_date(the_str):
    months_dict = {
            'jan': 1,
            'feb': 2,
            'mar': 3,
            'apr':4,
            'may':5,
            'jun':6,
            'jul':7,
            'aug':8,
            'sep':9,
            'oct':10,
            'nov':11,
            'dec':12
            }

    date = the_str.split(' ')
    month=months_dict[date[1].lower()]
    day=date[2]
    time=date[3]
    year=date[5]
    formatted_time = datetime.datetime.strptime(f'{year}-{str(month).zfill(2)}-{str(day).zfill(2)}-{time}','%Y-%m-%d-%H:%M:%S')
    return formatted_time




In [46]:
import os
data_list = os.listdir('./Scrapped_weibo/')

In [55]:
#### strategy one: take mean of all influencers
all_ = []
for index,data_name in tqdm(enumerate(data_list),total=len(data_list)):
    # if index<64:
    #     continue
    try:
        data = pickle.load(open(f'./Scrapped_weibo/{data_name}','rb'))
        name = data[0]['user_name']
        dates =[ get_date(i['created_at']) for i in data]
        text = [i['text'] for i in data]
        res = encoder.encode(text)
        responses = [i['reposts_count'] for i in data]
        comments_count = [i['comments_count'] for i in data]
        attitudes_count = [i['attitudes_count'] for i in data]
        features = pd.DataFrame({
            'date':dates,
            'responses':responses,
            'comments_count':comments_count,
            'attitudes_count':attitudes_count,
            'influencer':[name]*len(dates)
        })

        for i in range(res.shape[1]):
            features[f's{i}']=res[:,i]

        all_.append(features)
    except Exception as e:
        print(data_name,e)
        continue
    

88it [26:02, 17.76s/it] 


In [76]:
all_df = pd.concat(all_)

In [57]:
all_df.to_csv('features.csv',index=False)

In [80]:
#### 9:30 - 15:00 each day
### only take 15:00 - 9:30 (next day) data into account
### only data since 2018-01-01
all_df['time'] = all_df.date.dt.time
all_df['day'] = all_df.date.dt.date
all_df['year'] = all_df.date.dt.year
all_df['DOW'] = all_df.date.dt.day_of_week
all_df = all_df[all_df.year>=2018]
unique_date = sorted(all_df.date.dt.date.unique())


In [146]:
mean_res = {}
for date in unique_date:
    DOW = pd.DataFrame([pd.to_datetime(date)])[0].dt.day_of_week.values[0]
    if DOW==1:
        sub=all_df[(all_df.date>=datetime.datetime.strptime(
            str(date-datetime.timedelta(days=2))+'-15:00:00', '%Y-%m-%d-%H:%M:%S'
            )) & (all_df.date<=datetime.datetime.strptime(
            str(date)+'-9:00:00', '%Y-%m-%d-%H:%M:%S'
                ))
                ]
    else:
        sub=all_df[(all_df.date>=datetime.datetime.strptime(
        str(date-datetime.timedelta(days=1))+'-15:00:00', '%Y-%m-%d-%H:%M:%S'
        )) & (all_df.date<=datetime.datetime.strptime(
        str(date)+'-9:00:00', '%Y-%m-%d-%H:%M:%S'
            ))
            ]

    if len(sub)<5:
        continue
    if len(sub.influencer.unique())<5:
        continue

    mean_features = sub.groupby('influencer').mean()
    ind_count = mean_features.shape[0]
    mean_mean_features = mean_features[[f's{i}' for i in range(0,768)]].mean(axis=0)

    mean_mean_features={
        **{'ind_count':ind_count},
        **dict(mean_mean_features)

    }

    mean_res[date] = mean_mean_features
    

In [147]:
all_mean_features = pd.DataFrame(mean_res).T.reset_index(drop=False).rename(columns={'index':'date'})

In [148]:
all_mean_features.to_csv('mean_features.csv',index=False)